In [ ]:
"""
A Pythonic implementation of a MultiLayer Perceptron
with back propogation of error and stochastic gradient descent.


"""

In [24]:
%matplotlib inline

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from scipy import optimize

import time
import shutil
import zipfile
import os

import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D


from urllib import urlretrieve
from datetime import datetime

np.set_printoptions(precision=4)
np.set_printoptions(threshold=5)
np.set_printoptions(suppress=True)

pd.set_option('precision', 3, 'notebook_repr_html', True)

sns.set_style("darkgrid", {"grid.linewidth": .5, "axes.facecolor": ".9"})

sns.set(rc={"figure.figsize": (6, 6)})
np.random.seed(sum(map(ord, "palettes")))

# Get The Data

In [4]:
def get_dataset(url, data='data'):
    """
    Get's the data set necessary to train the ANN.
    """
    filename = url.rsplit("/")[-1]
    
    if not os.path.isdir(data): os.mkdir(data)

    urlretrieve(url, filename)

    compressed = zipfile.ZipFile(filename)
    compressed.extractall(data)
    compressed.close()
    
    os.remove(filename)

# The Student Performance Data

In [5]:
data = 'data'
url  = 'http://archive.ics.uci.edu/ml/machine-learning-databases/00320/student.zip'

get_dataset(url, data)

df_mat = pd.read_csv('data/student-mat.csv', sep=';')
df_por = pd.read_csv('data/student-por.csv', sep=';')

# Seperate Training and Testing Set

In [6]:
def assign_to_set(df):

    samples = np.random.choice(df.index,
                               size=np.int64(np.ceil(df.index.size * 0.2)),
                               replace=False,
    )
    df.ix[samples, 'testing'] = True

    return df

In [7]:
df_mat['testing'] = False

grouped = df_mat.groupby('school', group_keys=False).apply(assign_to_set)

df_mat_train = df_mat[grouped.testing == False]
df_mat_test  = df_mat[grouped.testing == True]

df_mat_train.to_csv('data/student_math_training.csv')
df_mat_test.to_csv('data/student_math_test.csv')

trainX = np.array(df_mat_train[['goout', 'studytime']])
trainY = np.array(df_mat_train[['G1']])

# Artificial Neural Network Implementation

In [8]:
class NeuralNet:
    
    def __init__(self):        
        #Define Hyperparameters
        self.inputLayerSize = 2
        self.outputLayerSize = 1
        self.hiddenLayerSize = 3
        
        #Weights (parameters)
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self, X):
        #Propogate inputs though network
        self.z2 = np.dot(X, self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2, self.W2)
        yHat = self.sigmoid(self.z3) 
        return yHat
        
    def sigmoid(self, z):
        #Apply sigmoid activation function to scalar, vector, or matrix
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        #Gradient of sigmoid
        return np.exp(-z)/((1+np.exp(-z))**2)
    
    def costFunction(self, X, y):
        #Compute cost for given X,y, use weights already stored in class.
        self.yHat = self.forward(X)
        J = 0.5*sum((y-self.yHat)**2)
        return J
        
    def costFunctionPrime(self, X, y):
        #Compute derivative with respect to W and W2 for a given X and y:
        self.yHat = self.forward(X)
        
        delta3 = np.multiply(-(y-self.yHat), self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T, delta3)
        
        delta2 = np.dot(delta3, self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(X.T, delta2)  
        
        return dJdW1, dJdW2
    
    #Helper Functions for interacting with other classes:
    def getParams(self):
        #Get W1 and W2 unrolled into vector:
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        #Set W1 and W2 using single paramater vector.
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, X, y):
        dJdW1, dJdW2 = self.costFunctionPrime(X, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

<h3 align = 'center'> Variables </h3>

|Code Symbol | Math Symbol | Definition | Dimensions
| :-: | :-: | :-: | :-: |
|X|$$X$$|Input Data, each row in an example| (numExamples, inputLayerSize)|
|y |$$y$$|target data|(numExamples, outputLayerSize)|
|W1 | $$W^{(1)}$$ | Layer 1 weights | (inputLayerSize, hiddenLayerSize) |
|W2 | $$W^{(2)}$$ | Layer 2 weights | (hiddenLayerSize, outputLayerSize) |
|z2 | $$z^{(2)}$$ | Layer 2 activation | (numExamples, hiddenLayerSize) |
|a2 | $$a^{(2)}$$ | Layer 2 activity | (numExamples, hiddenLayerSize) |
|z3 | $$z^{(3)}$$ | Layer 3 activation | (numExamples, outputLayerSize) |
|J | $$J$$ | Cost | (1, outputLayerSize) |

# Double Check Gradient

In [9]:
def computeNumericalGradient(N, X, y):
    paramsInitial = N.getParams()
    numgrad = np.zeros(paramsInitial.shape)
    perturb = np.zeros(paramsInitial.shape)
    e = 1e-4

    for p in range(len(paramsInitial)):
        #Set perturbation vector
        perturb[p] = e
        N.setParams(paramsInitial + perturb)
        loss2 = N.costFunction(X, y)

        N.setParams(paramsInitial - perturb)
        loss1 = N.costFunction(X, y)

        #Compute Numerical Gradient
        numgrad[p] = (loss2 - loss1) / (2*e)

        #Return the value we changed to zero:
        perturb[p] = 0

        #Return Params to original value:
        N.setParams(paramsInitial)

        return numgrad 

# Instantiate and Set Data

In [10]:
class Trainer:
    
    def __init__(self, N):
        #Make Local reference to network:
        self.N = N
        
    def callbackF(self, params):
        self.N.setParams(params)
        self.J.append(self.N.costFunction(self.X, self.y))   
        
    def costFunctionWrapper(self, params, X, y):
        self.N.setParams(params)
        cost = self.N.costFunction(X, y)
        grad = self.N.computeGradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        #Make an internal variable for the callback function:
        self.X = X
        self.y = y

        #Make empty list to store costs:
        self.J = []
        
        params0 = self.N.getParams()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.costFunctionWrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callbackF)

        self.N.setParams(_res.x)
        self.optimizationResults = _res

#Plot projections of our new data:

# Distribution

In [11]:
trainX = np.array(df_mat_train[['goout', 'studytime']])
trainY = np.array(df_mat_train[['G1']])

In [12]:
fig = plt.figure(0,(8,3))

plt.subplot(1,2,1)
plt.scatter(trainX[:,0], trainY)
plt.grid(1)
plt.xlabel('Hours Studying')
plt.ylabel('Test Score')

plt.subplot(1,2,2)
plt.scatter(trainX[:,1], trainY)
plt.grid(1)
plt.xlabel('Hours Outside')
plt.ylabel('Test Score')



#Normalize

# Train network with new data:

In [13]:
NN = NeuralNet()
T  = Trainer(NN)

T.train(trainX, trainY)

hoursSleep = np.linspace(0, 10, 100)
hoursStudy = np.linspace(0, 5, 100)

hoursSleepNorm = hoursSleep/10.
hoursStudyNorm = hoursStudy/5.

a, b  = np.meshgrid(hoursSleepNorm, hoursStudyNorm)

allInputs = np.zeros((a.size, 2))
allInputs[:, 0] = a.ravel()
allInputs[:, 1] = b.ravel()


allOutputs = NN.forward(allInputs)

yy = np.dot(hoursStudy.reshape(100,1), np.ones((1,100)))
xx = np.dot(hoursSleep.reshape(100,1), np.ones((1,100))).T

CS = plt.contour(xx,yy,100*allOutputs.reshape(100, 100))

plt.clabel(CS, inline=1, fontsize=10)
plt.xlabel('Hours Go Out')
plt.ylabel('Hours Studying')

fig = plt.figure()

ax = fig.gca(projection='3d')

surf = ax.plot_surface(xx, yy, 100*allOutputs.reshape(100, 100), cmap=plt.cm.jet)

ax.set_xlabel('Hours Go Out')
ax.set_ylabel('Hours Studying')
ax.set_zlabel('Test Score')

Optimization terminated successfully.
         Current function value: 16971.500000
         Iterations: 1
         Function evaluations: 2
         Gradient evaluations: 2
